# Imports 

In [47]:
import pandas as pd
import numpy as np
import os

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Read in data and make sure relevant columns are string/character 

- San Diego data: `naics_code` and `account_key`
- NAICS details data: `naics` 

Run code below; if pulling from github, pathname should be fine; if working elsewhere may need to edit path name at read in 

In [48]:
# uncomment if u need to find your path: os.getcwd()
SD = sd_df = pd.read_csv("../public_data/sd_df.csv")
NA = naics_df = pd.read_csv("../public_data/naics_df.csv")

In [49]:
cols_sd_use = ["naics_code", "account_key"]
cols_naics_use = ["naics"]

sd_df[cols_sd_use] = sd_df[cols_sd_use].astype(str)
naics_df[cols_naics_use] = naics_df[cols_naics_use].astype(str)

sd_df.dtypes
naics_df.dtypes
sd_df.head()
naics_df.head()

account_key          object
dba_name             object
council_district     object
naics_code           object
naics_description    object
naics_nchar           int64
dtype: object

naics                object
naics_description    object
dtype: object

,account_key,dba_name,council_district,naics_code,naics_description,naics_nchar
0,1974000448,ERNST & YOUNG LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6
1,1974011093,HECHT SOLBERG ROBINSON GOLDBERG & BAGLEY LLP,cd_3,5411,LEGAL SERVICES,4
2,1978039819,RSM US LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6
3,1978042092,THORSNES BARTOLOTTA MCGUIRE LLP,cd_3,5411,LEGAL SERVICES,4
4,1979046817,KORENIC & WOJDOWSKI LLP,cd_7,5412,ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES,4


,naics,naics_description
0,111140,Wheat Farming
1,111160,Rice Farming
2,111150,Corn Farming
3,111110,Soybean Farming
4,111120,Oilseed (except Soybean) Farming


## "Inner join"- retain only San Diego businesses with details on their NAICS code

- Use the `naics_code` column in the San Diego business data as the join key
- Use the `naics` column in the NAICS code details data as the join key

- Do an inner join of the San Diego data onto the NAICS code details using these join keys
- After the inner join, print some examples of San Diego businesses lost in the merge
- Use value_counts() on the `naics_nchar` column in the San Diego data to see why they might have gotten lost


In [55]:
# your code here
df_join = naics_df.merge(sd_df, right_on = "naics_code", left_on="naics", how="inner")
df_join.head()
sd_df["naics_nchar"].value_counts()
df_join["naics_nchar"].value_counts()

,naics,naics_description_x,account_key,dba_name,council_district,naics_code,naics_description_y,naics_nchar,naics_code_pad
0,311999,All Other Miscellaneous Food Manufacturing,2020007786,PRIMAL COFFEE COMPANY,cd_2,311999,ALL OTHER MISCELLANEOUS FOOD MFG,6,311999
1,311999,All Other Miscellaneous Food Manufacturing,2020007786,PRIMAL COFFEE COMPANY,cd_2,311999,ALL OTHER MISCELLANEOUS FOOD MFG,6,311999
2,311999,All Other Miscellaneous Food Manufacturing,2020007786,PRIMAL COFFEE COMPANY,cd_2,311999,ALL OTHER MISCELLANEOUS FOOD MFG,6,311999
3,311999,All Other Miscellaneous Food Manufacturing,2020007786,PRIMAL COFFEE COMPANY,cd_2,311999,ALL OTHER MISCELLANEOUS FOOD MFG,6,311999
4,311999,All Other Miscellaneous Food Manufacturing,2020007786,PRIMAL COFFEE COMPANY,cd_2,311999,ALL OTHER MISCELLANEOUS FOOD MFG,6,311999


naics_nchar
5    150
4     89
6     49
3     42
2     23
Name: count, dtype: int64

naics_nchar
6    52
Name: count, dtype: int64

## "Left join"- retain all sd businesses even if naics code isn't in `naics_df`

- Using the same join keys as above, and treating the San Diego businesses as the left hand side data, left join the naics code details onto the San Diego businesses
- Use the `indicator` argument within merge to create an indicator, `naics_merge_status`, to help with later merge diagnostics and examine sample of ones that didn't merge
- Use the `suffixes` argument within merge to add `_sd` as the left suffix, `_census` as the right suffix
- Use `naics_merge_status` in the merged result to look at a sample of San Diego businesses that weren't matched with `naics_df`

In [57]:
# your code here
df_left_join = sd_df.merge(naics_df, left_on = "naics_code", right_on="naics", how="left", suffixes=("_sd", "_census"), indicator="naics_merge_status")
df_left_join.head()
df_left_join[df_left_join["naics_merge_status"] != "both"].head()

,account_key,dba_name,council_district,naics_code,naics_description_sd,naics_nchar,naics_code_pad,naics,naics_description_census,naics_merge_status
0,1974000448,ERNST & YOUNG LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6,541211,541211,Offices of Certified Public Accountants,both
1,1974011093,HECHT SOLBERG ROBINSON GOLDBERG & BAGLEY LLP,cd_3,5411,LEGAL SERVICES,4,541100,NaN,NaN,left_only
2,1978039819,RSM US LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6,541211,541211,Offices of Certified Public Accountants,both
3,1978042092,THORSNES BARTOLOTTA MCGUIRE LLP,cd_3,5411,LEGAL SERVICES,4,541100,NaN,NaN,left_only
4,1979046817,KORENIC & WOJDOWSKI LLP,cd_7,5412,ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES,4,541200,NaN,NaN,left_only


,account_key,dba_name,council_district,naics_code,naics_description_sd,naics_nchar,naics_code_pad,naics,naics_description_census,naics_merge_status
1,1974011093,HECHT SOLBERG ROBINSON GOLDBERG & BAGLEY LLP,cd_3,5411,LEGAL SERVICES,4,541100,NaN,NaN,left_only
3,1978042092,THORSNES BARTOLOTTA MCGUIRE LLP,cd_3,5411,LEGAL SERVICES,4,541100,NaN,NaN,left_only
4,1979046817,KORENIC & WOJDOWSKI LLP,cd_7,5412,ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES,4,541200,NaN,NaN,left_only
5,1979053082,GRIMM VRANJES GREER STEPHAN & BRIDGMAN LLP,cd_3,5411,LEGAL SERVICES,4,541100,NaN,NaN,left_only
6,1981000840,BENINK & SLAVENS LLP,cd_7,5411,LEGAL SERVICES,4,541100,NaN,NaN,left_only


## Use group by and agg to see if there are differences in merge rates by area

- Using the left-joined dataframe created in previous step, create a boolean indicator, `is_lost`, that equals `True` if the merge indicator is equal to "left_only" (and is otherwise false)
- Find the proportion of businesses by council district that were lost in the left join (SD businesses that failed to match to `naics_df`). To do this, group by `council_district` and use the shortcut of taking the mean of the `is_lost` indicator to find the proportions lost by `council_district`. 

In [53]:
# your code here
is_lost = df_left_join["_merge"] == "left_only"
df_left_join["is_lost"] = is_lost
res = df_left_join.groupby("council_district")["is_lost"].mean()
res


council_district
cd_1    0.880597
cd_2    0.804348
cd_3    0.812500
cd_4    1.000000
cd_5    0.916667
cd_6    0.859649
cd_7    0.918919
cd_8    0.933333
cd_9    1.000000
Name: is_lost, dtype: float64

## Optional challenge exercise: add lagging 0's and see if merge rate improves

You noticed earlier that a big reason for non-matches is that the San Diego tax certificate NAICS codes were often less than six digits long, while the Census ones were always 6 digits.

You wonder if this is an issue where 0's in some of the SD's data naics codes got cut off (eg 540000 became 54), and if so whether adding these lagging zeros would improve the merge rate in a left join.

- Pad the `naics_code` column in `sd_df` with 0's to get that column up to 6-digits, using one of two approaches: 
  1. `str.pad` in pandas (https://pandas.pydata.org/docs/reference/api/pandas.Series.str.pad.html)
  2. for more of a challenge, write your own function! It should check the # of digits in the naics code string and pad it with 0's at the end up to 6 digits. To execute your function, use row-wise apply: `df.apply(lambda row: funcname(row.column), axis=1)`.
- Perform the same left join as above and see how the match rate changes
- Create an indicator variable--`is_new_match`---for new matches under the padded NAICS code; compare the `naics_description` column from San Diego versus Census in the new dataset for a sample of these new matches and comment on whether the padding seems to be correct

In [63]:
# your code here
sd_df["naics_code_pad"] = sd_df["naics_code"].str.pad(width=6, side='right', fillchar='0')

df_left_pad = sd_df.merge(naics_df, left_on = "naics_code_pad", right_on="naics", how="left", suffixes=("_sd", "_census"), indicator=True)
df_left_pad.head()
is_new_match = df_left_pad["naics_description_sd"].str.lower() == df_left_pad["naics_description_census"].str.lower()
df_left_pad[is_new_match].sample(n=20)
# padding seems to be correct! 

,account_key,dba_name,council_district,naics_code,naics_description_sd,naics_nchar,naics_code_pad,naics,naics_description_census,_merge
0,1974000448,ERNST & YOUNG LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6,541211,541211,Offices of Certified Public Accountants,both
1,1974011093,HECHT SOLBERG ROBINSON GOLDBERG & BAGLEY LLP,cd_3,5411,LEGAL SERVICES,4,541100,NaN,NaN,left_only
2,1978039819,RSM US LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6,541211,541211,Offices of Certified Public Accountants,both
3,1978042092,THORSNES BARTOLOTTA MCGUIRE LLP,cd_3,5411,LEGAL SERVICES,4,541100,NaN,NaN,left_only
4,1979046817,KORENIC & WOJDOWSKI LLP,cd_7,5412,ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES,4,541200,NaN,NaN,left_only


,account_key,dba_name,council_district,naics_code,naics_description_sd,naics_nchar,naics_code_pad,naics,naics_description_census,_merge
234,2015048818,ALPHA SQUARE APARTMENTS,cd_7,53139,OTHER ACTIVITIES RELATED TO REAL ESTATE,5,531390,531390,Other Activities Related to Real Estate,both
213,2014027828,STEES WALKER COMPANY LLP,cd_5,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6,541211,541211,Offices of Certified Public Accountants,both
13,1989013971,TIC TAC TOW LLP,cd_8,48841,MOTOR VEHICLE TOWING,5,488410,488410,Motor Vehicle Towing,both
365,2020006547,ROCKEYS TOOLS,cd_6,45439,OTHER DIRECT SELLING ESTABLISHMENTS,5,454390,454390,Other Direct Selling Establishments,both
286,2018024376,THRIVE AFFORDABLE VET CARE,cd_6,54194,VETERINARY SERVICES,5,541940,541940,Veterinary Services,both
289,2018027955,DR BOYDS PET RESORT,cd_3,54194,VETERINARY SERVICES,5,541940,541940,Veterinary Services,both
158,2008016723,CASEY GERRY SCHENK FRANCAVILLA BLATT & PENFIELD,cd_3,54111,OFFICES OF LAWYERS,5,541110,541110,Offices of Lawyers,both
108,2006010738,WENTHUR LAW GROUP LLP,cd_2,541199,ALL OTHER LEGAL SERVICES,6,541199,541199,All Other Legal Services,both
419,2021002956,THRIVE AFFORDABLE VET CARE,cd_5,54194,VETERINARY SERVICES,5,541940,541940,Veterinary Services,both
83,2005001749,EDWARD JONES,cd_7,52312,SECURITIES BROKERAGE,5,523120,523120,Securities Brokerage,both
